In [1]:
import numpy as np    
import pandas as pd    
import matplotlib.pyplot as plt  
import os  
import cartopy.crs as ccrs  
import cartopy.feature as cfeature  
import matplotlib.ticker as mticker  
from matplotlib.ticker import MultipleLocator, FuncFormatter

In [2]:
def dms_to_decimal(dms):
    """将度分秒转换为十进制度数"""
    dms = dms.replace('°', ' ').replace('′', ' ').replace('″', ' ').replace('"E', ' ').replace('"N', ' ').replace('"S', ' ')
    parts = dms.split()
    degrees = float(parts[0])
    minutes = float(parts[1]) if len(parts) > 1 else 0
    seconds = float(parts[2]) if len(parts) > 2 else 0
    decimal = degrees + (minutes / 60) + (seconds / 3600)
    return decimal

In [3]:
import xarray as xr
ds = xr.open_dataset(r"E:\all_draw\data\Copernicus_bathy\cmems_mod_glo_phy_my_0.083deg_static_1720691864705.nc")
ds

<xarray.Dataset>
Dimensions:    (latitude: 2041, longitude: 4320)
Coordinates:
  * latitude   (latitude) float32 -80.0 -79.92 -79.83 ... 89.83 89.92 90.0
  * longitude  (longitude) float32 -180.0 -179.9 -179.8 ... 179.8 179.8 179.9
Data variables:
    deptho     (latitude, longitude) float32 ...
Attributes:
    Conventions:       CF-1.11
    title:             Bathymetry
    institution:       Mercator Ocean
    producer:          CMEMS - Global Monitoring and Forecasting Centre
    source:            GLORYS12V1
    credit:            E.U. Copernicus Marine Service Information (CMEMS)
    contact:           servicedesk.cmems@mercator-ocean.eu
    references:        http://marine.copernicus.eu
    subset:source:     ARCO data downloaded from the Marine Data Store using ...
    subset:productId:  GLOBAL_MULTIYEAR_PHY_001_030
    subset:datasetId:  cmems_mod_glo_phy_my_0.083deg_static_202311--ext--bathy
    subset:date:       2024-07-11T09:57:44.706Z

In [4]:
elevation = ds['deptho'].values
# elevation = elevation[::100,::100]
lon = ds['longitude'].values
# lon = lon[::100]
lat = ds['latitude'].values
# lat = lat[::100]

In [5]:
# 自定义刻度格式化函数
def lon_formatter(x, pos):
    if x < 0:
        return f'{abs(int(x))}°W'
    elif x > 180:
        return f'{360-int(x)}°W'
    else:
        return f'{int(x)}°E'

def lat_formatter(x, pos):
    if x > 0:
        return f'{int(x)}°N' 
    else:
        return f'{int(x)}°S'

In [8]:
stations = os.listdir(r'D:\ze_1112\最新数据\CTD补上溶解氧_04共享航次计划调查资料汇交表')
plt.rcParams['font.sans-serif'] = ['SimHei']  # 用来正常显示中文标签
plt.rcParams['axes.unicode_minus'] = False  # 用来正常显示负号
plt.rcParams['font.size'] = 22
fig = plt.figure(figsize=(10, 10))
ax = fig.add_subplot(111, projection=ccrs.PlateCarree())
ax.set_extent([125, 170, -5, 30], crs=ccrs.PlateCarree())
# 添加填充
ax.add_feature(cfeature.BORDERS, linestyle=':')
ax.add_feature(cfeature.COASTLINE, linewidth=0.5)
ax.add_feature(cfeature.LAND, zorder=0, edgecolor='black')
# ax.add_feature(cfeature.OCEAN, zorder=0, edgecolor='black')
# 绘制站点位置
for i in stations:
        df = pd.read_excel(r'D:\ze_1112\最新数据\CTD补上溶解氧_04共享航次计划调查资料汇交表\%s' % i, skiprows=1, header=1)
        df = df.drop(0)
        longitude = df['经度'].apply(dms_to_decimal).values # type: ignore
        latitude = df['纬度'].apply(dms_to_decimal).values  # type: ignore
        name = df['站位名'].values  
        print(longitude[3], latitude[3], name[3])
        ax.scatter(longitude[3], latitude[3], s=20, c='r', marker='o', label=name[3], transform=ccrs.PlateCarree(), zorder=10)
        # ax.text(longitude[3], latitude[3], name[3], fontsize=5, transform=ccrs.PlateCarree(), zorder=11)
        # ax.scatter(longitude, latitude, s=10, c='r', marker='o', label=name, transform=ccrs.PlateCarree(), zorder=10)
# 绘制海岸线
ax.add_feature(cfeature.RIVERS, zorder=0, edgecolor='black')
cf = ax.contourf(lon, lat, elevation, levels=20, cmap='terrain', transform=ccrs.PlateCarree())
ax.spines['geo'].set_linewidth(1.5)
ax.spines['geo'].set_edgecolor('black')
ax.set_xlabel('Longitude')
ax.set_ylabel('Latitude')
gl = ax.gridlines(draw_labels=False, linewidth=1, color='white', alpha=0.15, linestyle='--')
gl.top_labels = False
gl.right_labels = False
gl.xlabel_style = {'size': 15, 'color': 'black'}
gl.ylabel_style = {'size': 15, 'color': 'black'}
ax.set_xticks(np.arange(125, 170, 10))
ax.set_yticks(np.arange(-5, 30, 5))
ax.xaxis.set_major_formatter(FuncFormatter(lon_formatter))
ax.yaxis.set_major_formatter(FuncFormatter(lat_formatter))

# plt.show()
plt.savefig('stations111.png', dpi=300, bbox_inches='tight')
plt.close()

130.00045555555556 19.99903888888889 E130-02
129.99689444444442 19.00379722222222 E130-03
130.0005 17.99888888888889 E130-04
130.00107222222223 16.999380555555557 E130-05
130.00155555555557 16.000186111111113 E130-06
129.98307222222223 15.001838888888889 E130-07
129.99995 13.998577777777777 E130-08
129.9987222222222 13.00013888888889 E130-09
129.9998222222222 12.499580555555555 E130-09s
129.9863 11.993338888888887 E130-10
129.98491944444444 10.998058333333333 E130-11
129.98883888888886 10.496827777777776 E130-11s
130.00091944444443 10.001091666666667 E130-12
129.9978611111111 8.998880555555555 E130-13
130.0007611111111 7.999491666666667 E130-14
139.99968055555553 1.5082805555555556 E140-01
139.99952777777776 1.7506194444444445 E140-02
139.99938055555555 2.0003611111111113 E140-03orQB01
139.99852222222222 2.25085 E140-04
139.99834444444443 2.500075 E140-05
139.99991944444443 2.749661111111111 E140-06
140.00206111111112 3.0023694444444446 E140-07
139.99511944444444 3.2415805555555557 E14